Importing Liabraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import resample
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

Data Reading

In [14]:
# Define the file path for the Excel file stored in Google Drive
# file_path = 'DataSet_EU_3k_5k.xlsx'
file_path = 'Test.xlsx'

# Use pandas to read the Excel file located at the specified file path
data = pd.read_excel(file_path)



In [15]:
# Display the first few rows of the dataframe to get an overview of the data
data.head()

,GSNR_35,GSNR_36,GSNR_37,GSNR_38,GSNR_39,GSNR_40,GSNR_41,GSNR_42,GSNR_43,GSNR_44,...,GSNR_69,GSNR_70,GSNR_71,GSNR_72,GSNR_73,GSNR_74,GSNR_75,GSNR_76,No. Spans,Total Distance(m)
0,77.575997,0.000000,0.00000,83.989788,0.000000,86.062347,0.000000,0.000000,0.000000,0.000000,...,90.061284,0.000000,0.000000,0.000000,0.000000,95.045789,0.000000,0.000000,8,690608
1,78.279541,80.219954,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,92.560867,0.000000,0.000000,89.544975,0.000000,0.000000,0.000000,8,690608
2,0.000000,72.294523,73.59654,0.000000,0.000000,70.952966,68.330300,65.326872,67.614572,71.283118,...,0.000000,0.000000,90.991977,0.000000,0.000000,0.000000,85.512399,83.114113,8,690608
3,0.000000,0.000000,0.00000,0.000000,88.056368,0.000000,86.757766,0.000000,0.000000,0.000000,...,76.409583,73.664915,74.651700,79.071217,0.000000,0.000000,0.000000,0.000000,8,690608
4,0.000000,0.000000,0.00000,83.209197,0.000000,77.030461,74.502742,0.000000,74.902228,71.293391,...,0.000000,0.000000,78.299438,74.910765,72.030907,73.790264,81.190439,0.000000,8,690608


In [16]:
# Define the attribute columns and the target variable
attribute_columns = [f'Power_{i}' for i in range(1, 77)] + [f'ASE_{i}' for i in range(1, 77)] + [f'NLI_{i}' for i in range(1, 77)]+ ['No. Spans'] + ['Total Distance(m)']

# Label
target_column = 'GSNR_1'

In [24]:
# MinMax Normalization
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize the attribute columns using MinMaxScaler and convert the result back to a DataFrame
x = scaler.fit_transform(data[attribute_columns])
x = pd.DataFrame(x, columns=attribute_columns)

# Extract the target variable values and reshape them for normalization
# y = data[target_column].values.reshape(-1, 1)
# Normalize the target variable using MinMaxScaler and convert the result back to a DataFrame
y = scaler.fit_transform(data[target_column])
y = pd.DataFrame(y, columns=[target_column])

# Display the first few rows of the normalized data for selected columns
print("MinMax Normalized Data:")
print(x['Power_1'].head(), x['ASE_1'].head(), x['NLI_1'].head(), x['No. Spans'].head(), x['Total Distance(m)'].head())
print(y.head())

C:\Users\rafia\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
C:\Users\rafia\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


KeyError: 'GSNR_1'

In [ ]:
from sklearn.model_selection import train_test_split
# Define the function to split data according to the specified pattern
def custom_train_test_split(data, labels, samples_per_block=3000, train_samples_per_block=2500, test_samples_per_block=500):
    # Initialize lists to hold the training and testing data and labels
    train_data = []
    test_data = []
    train_labels = []
    test_labels = []

    # Determine the total number of samples in the data
    total_samples = len(data)

    # Loop over the data in blocks of the specified size (samples_per_block)
    for start in range(0, total_samples, samples_per_block):
        # Define the end of the current block
        end = start + samples_per_block
        # Define the end of the training samples within the current block
        train_end = start + train_samples_per_block

        # Adjust train_end and end if they exceed the total number of samples
        if train_end > total_samples:
            train_end = total_samples
        if end > total_samples:
            end = total_samples

        # Append the training and testing data and labels for the current block to the respective lists
        train_data.append(data[start:train_end])
        test_data.append(data[train_end:end])
        train_labels.append(labels[start:train_end])
        test_labels.append(labels[train_end:end])

    # Concatenate the lists into DataFrames and reset the index
    train_data = pd.concat(train_data).reset_index(drop=True)
    test_data = pd.concat(test_data).reset_index(drop=True)
    train_labels = pd.concat(train_labels).reset_index(drop=True)
    test_labels = pd.concat(test_labels).reset_index(drop=True)

    # Return the training and testing data and labels
    return train_data, test_data, train_labels, test_labels

# Create a list of attribute columns excluding those that start with 'frequency'
# attribute_columns_without_frequency = [col for col in attribute_columns if not col.startswith('frequency')]

# Assuming 'x' and 'y' are your DataFrames for MinMax normalized data
# Perform the custom train-test split on the normalized data
# X_train, X_test, Y_train, Y_test = custom_train_test_split(x[attribute_columns_without_frequency], y[target_column])
X_train, X_test, Y_train, Y_test = train_test_split(
    x[attribute_columns], 
    y[target_column], 
    test_size=0.2,  # Equivalent to test_samples_per_block / samples_per_block
    random_state=42  # For reproducibility
)


# Display shapes to verify the split
print("Normalized data with equal path distribution in train and test data :\n")
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)
print("Train labels shape:", Y_train.shape)
print("Test labels shape:", Y_test.shape)

Normalized data with equal path distribution in train and test data :

Train data shape: (15000, 230)
Test data shape: (3000, 230)
Train labels shape: (15000,)
Test labels shape: (3000,)


In [ ]:
import pickle

# Save the data to a file
with open('data.pkl', 'wb') as f:
    pickle.dump((X_train, Y_train, X_test, Y_test), f)